In [2]:
%config Completer.use_jedi=False

In [3]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import os
from sklearn.model_selection import train_test_split

from ruslan_nn.schnet import SchNet
import pickle
import wandb

In [4]:
targets = pd.read_csv('ruslan_nn/properties16k.csv')
label=targets['energy_per_atom']

In [5]:
with open('ruslan_nn/structures16k.pickle', 'rb') as file:
    structures = pickle.load(file)

In [6]:
from torch_geometric.data import Data
import torch
import ase
from pymatgen.io.ase import AseAtomsAdaptor

i=0
data_atoms = []
for _id in tqdm(targets._id):
    atoms=AseAtomsAdaptor.get_atoms(structures[str(_id)])
    # set the atomic numbers, positions, and cell
    atom = torch.Tensor(atoms.get_atomic_numbers())
    positions = torch.Tensor(atoms.get_positions())
    natoms = positions.shape[0]
    
    # put the minimum data in torch geometric data object
    data = Data(
        pos=positions,
        z= atom,
       # natoms=natoms,
    )
    
    # calculate energy
    data.y = label[i]
    i=i+1
    data_atoms.append(data)

100%|██████████| 14718/14718 [00:25<00:00, 584.43it/s]


In [7]:
from torch_geometric.data import DataLoader
train_dataset, test_dataset = train_test_split(data_atoms, test_size=0.2)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)

In [13]:
wandb.init(
    project="schnet_dichalcogenides", entity="inno-materials-ai", save_code=True, name='baseline'
)

In [14]:
model=SchNet()
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
model = model.to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
epochs = 200

loss_func = torch.nn.L1Loss() #define loss
scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, epochs=epochs,
                                                steps_per_epoch=len(train_loader),
                                                max_lr=1e-3)

In [ ]:
for epoch in range(epochs):
    model.train()
    valid_loss=0
    train_loss=0

     #shuffle the training data each epoch
    for d in tqdm(train_loader): #go over each training point
        data = d.to(device)#send data to device
        out = model(data) 
        optimizer.zero_grad() #zero gradients
        #evaluate data point
        loss = loss_func(out.view(-1), data.y.view(-1)) #L1 error loss
         #add loss value to aggregate loss
        loss.backward() #compute gradients
        optimizer.step() #apply optimization
        scheduler.step()
        train_loss += loss.item()
    with torch.no_grad():
        model.eval()     # Optional when not using Model Specific layer
        for d in tqdm(test_loader):
            data = d.to(device)
            target = model(data)
            loss = loss_func(target.view(-1), data.y.view(-1))
            valid_loss += loss.item()
        
    print('Epoch: {:03d}, Average loss: {:.5f}'.format(epoch, train_loss/len(train_loader)))
    print('Epoch: {:03d}, Average loss: {:.5f}'.format(epoch, valid_loss/len(test_loader)))
    wandb.log({
        "train_mae": train_loss/len(train_loader),
        "test_mae": valid_loss/len(test_loader),
    })    
    

100%|██████████| 92/92 [00:12<00:00,  7.50it/s]


Epoch: 000, Average loss: 0.30561
Epoch: 000, Average loss: 0.04736


100%|██████████| 92/92 [00:12<00:00,  7.17it/s]


Epoch: 001, Average loss: 0.04068
Epoch: 001, Average loss: 0.03534


100%|██████████| 92/92 [00:12<00:00,  7.30it/s]


...

100%|██████████| 92/92 [00:04<00:00, 20.97it/s]


Epoch: 104, Average loss: 0.00932
Epoch: 104, Average loss: 0.00632


100%|██████████| 92/92 [00:04<00:00, 20.48it/s]


Epoch: 105, Average loss: 0.00947
Epoch: 105, Average loss: 0.01288


 28%|██▊       | 102/368 [00:11<00:29,  9.07it/s]

In [12]:
print(model)

SchNet(hidden_channels=128, num_filters=128, num_interactions=4, num_gaussians=50, cutoff=10.0)
